# Moniteurs de biais de modèles Amazon SageMaker : Équité et explicabilité avec SageMaker Clarify

Amazon SageMaker Clarify contribue à améliorer vos modèles de machine learning en détectant les biais potentiels et en expliquant comment ces modèles font des prédictions. La fonctionnalité d'équité et d'explicabilité fournie par SageMaker Clarify fait un pas en avant pour permettre aux clients d'AWS de créer des modèles de machine learning fiables et compréhensibles. Le produit est fourni avec les outils nécessaires pour vous aider dans les tâches suivantes.

- Mesurer les biais qui peuvent se produire à chaque étape du cycle de vie du ML (collecte des données, entraînement et réglage de modèle, et surveillance des modèles ML déployés pour l'inférence).
- Générer des rapports sur la gouvernance des modèles à l'intention des équipes chargées des risques et de la conformité, et des régulateurs externes.
- Fournir des explications sur les données, les modèles et la surveillance utilisés pour évaluer les prédictions.

## Imports

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### Configuration pour cette tâche

In [ ]:
# Replace below <bucket_name> with the value the modelArtifactBucket value at the left of the instruction page
bucket = '<bucket_name>'

# Get production endpoint name
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Get production model name
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

### Fichiers de données

Lisez les fichiers de données

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### Activer la capture de données sur votre point de terminaison

Pour activer la capture de données pour les tâches de surveillance, spécifiez ici la nouvelle option de capture appelée DataCaptureConfig. Celle-ci permet de capturer la charge utile de la demande et celle de la réponse du point de terminaison. La configuration de capture s'applique à toutes les variantes. 

Cette opération dure en général de 10 à 15 minutes.

**Remarque :** ceci peut être automatisé via cloudformation (reviewer - please suggest)

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Il est maintenant temps d'appliquer la nouvelle configuration et d'attendre qu'elle soit appliquée
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### Test du point de terminaison avec un échantillon de trafic

In [ ]:
# Ici, nous choisissons aléatoirement un sous-ensemble de données à partir des ensembles de données de test.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Invoke Endpoint with test data for a quick test
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## Consulter les données capturées

Listez maintenant les fichiers de capture de données stockés dans Amazon S3. Vous devez vous attendre à voir différents fichiers de différentes périodes organisés en fonction de l'heure à laquelle l'invocation a eu lieu. Le format du chemin d'accès Amazon S3 est le suivant :

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "waiting for the file to load..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        contine
print("Found Capture Files:")
print("\n ".join(capture_files))

Ensuite, examinez le contenu d'un seul fichier de capture. Ici, vous devriez voir toutes les données capturées dans un fichier au format JSON spécifique à Amazon SageMaker. Jetez un coup d'œil aux premières lignes du fichier capturé.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Enfin, le contenu d'une seule ligne est présenté ci-dessous dans un fichier au format JSON pour que vous puissiez l'observer un peu mieux.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Configuration d'Amazon SageMaker Clarify

**Remarque :** les exemples de code ci-dessous sont tirés de [sample code](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) fourni par Amazon SageMaker. Certains de ces codes ont été mis à jour pour répondre aux exigences de l'atelier.

In [ ]:
# Exécutez le processeur SageMaker Clarify à l'aide de la commande suivante
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

Téléchargez les jeux de données de test et d'entraînement dans S3. _train_uri_ et _test_uri_ seront utilisés pour extraire directement les données de S3 en écrivant dataconfig et modelconfig dans les étapes suivantes.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

### Détection de biais

SageMaker Clarify vous aide à détecter les éventuels biais de pré- et de post-entraînement à l'aide de différentes métriques.

### Écriture de DataConfig et ModelConfig

Un objet **DataConfig** communique à SageMaker Clarify certaines informations de base sur les I/O de données. Vous indiquerez où trouver le jeu de données d'entrée, où stocker la sortie, la colonne cible (étiquette), les noms d'en-tête et le type de jeu de données.

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

Un objet **ModelConfig** communique des informations sur votre modèle entraîné. Pour éviter tout trafic supplémentaire vers vos modèles de production, SageMaker Clarify configure et supprime un point de terminaison dédié lors du traitement.

- **instance_type** et **instance_count** indiquent le type d'instance et le nombre d'instances sur lesquels vous préférez que votre modèle soit exécuté pendant le traitement par SageMaker Clarify. Le jeu de données de test étant petit, une seule instance standard suffit pour exécuter cet exemple. Si vous avez un grand jeu de données complexes, vous pouvez utiliser un meilleur type d'instance pour accélérer, ou ajouter plus d'instances pour permettre la parallélisation de Spark.

- **accept_type** indique le format de charge utile de la réponse du point de terminaison, et **content_type** le format de charge utile de la demande au point de terminaison.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

Un **ModelPredictedLabelConfig** fournit des informations sur le format de vos prédictions. Le modèle Random Forest produit des probabilités pour les échantillons. SageMaker Clarify invoque donc le point de terminaison, puis utilise probability_threshold pour convertir la probabilité en étiquettes binaires pour l'analyse des biais. Une prédiction supérieure au seuil est interprétée comme une valeur d'étiquette 1 et une prédiction inférieure ou égale comme une valeur d'étiquette 0.

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### Écriture de BiasConfig
SageMaker Clarify a également besoin d'informations sur ce que sont les colonnes sensibles (facets), ce que peuvent être les caractéristiques sensibles (facet_values_or_threshold) et ce que sont les résultats souhaitables (label_values_or_threshold). SageMaker Clarify peut traiter des données catégorielles et continues pour facet_values_or_threshold et pour label_values_or_threshold. Dans ce cas, nous utilisons des données catégorielles.

Nous spécifions cette information dans l'API BiasConfig. Ici, le résultat positif est Churn = 0, _Account Length_ est une catégorie sensible, et les répondants _Day Calls_ sont le groupe sensible. group_name est utilisé pour entraîner des sous-groupes pour la mesure.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**Biais de pré-entraînement**

Un biais peut être présent dans vos données avant tout entraînement de modèle. L'inspection de vos données avant le début de l'entraînement peut vous aider à détecter toute lacune dans la collecte des données, à informer votre ingénierie des fonctionnalités et à comprendre les biais sociétaux que les données peuvent refléter.

Le calcul des métriques de biais de pré-entraînement ne nécessite pas un modèle entraîné.

**Biais de post-entraînement**

Le calcul des métriques de biais de post-entraînement ne nécessite pas un modèle entraîné.

Des données d'entraînement non biaisées (déterminées par les concepts d'équité mesurés par la métrique du biais) peuvent néanmoins donner lieu à des prédictions de modèle biaisées après entraînement. Ce résultat dépend de plusieurs facteurs, dont le choix des hyperparamètres.

Vous pouvez exécuter ces options séparément avec run_pre_training_bias() et run_post_training_bias() ou en même temps avec run_bias() comme indiqué ci-dessous.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**Consultation du rapport de biais**

Dans Studio, vous pouvez consulter les résultats sous l'onglet des expériences.

<img src="./recordings/bias_report.gif">

Chaque métrique de biais est accompagnée d'explications détaillées et d'exemples que vous pouvez explorer.

<img src="./recordings/bias_detail.gif">

Vous pouvez également résumer les résultats dans un tableau pratique !

<img src="./recordings/bias_report_chart.gif">



Si vous n'êtes pas encore un utilisateur de Studio, vous pouvez accéder au rapport de biais aux formats pdf, html et ipynb dans le compartiment S3 suivant :

In [ ]:
bias_report_output_path

### Explication des prédictions

Il existe des besoins commerciaux en expansion et des réglementations législatives qui exigent des explications sur les raisons pour lesquelles un modèle a pris une certaine décision. SageMaker Clarify utilise SHAP pour expliquer la contribution de chaque caractéristique d'entrée à la décision finale.

L'algorithme Kernel SHAP nécessite une base de référence (également appelée jeu de données d'arrière-plan). Le type de jeu de données de base doit être identique au dataset_type de DataConfig, et les échantillons de base ne doivent comprendre que des caractéristiques. Par définition, la base de référence doit être soit un URI S3 vers le fichier de données de base, soit une liste d'échantillons sur place. Dans ce cas, nous avons choisi la seconde option et avons ajouté le premier échantillon de l'ensemble de données de test à la liste.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### Consultation du rapport d'explicabilité
Comme pour le rapport de biais, vous pouvez consulter le rapport d'explicabilité dans Studio sous l'onglet Expériences


<img src="./recordings/explainability_detail.gif">

L'onglet Informations sur le modèle contient des liens directs vers le rapport et des informations sur le modèle.

Si vous n'êtes pas encore un utilisateur de Studio, vous pouvez accéder à ce rapport de biais dans le compartiment S3 suivant :

In [ ]:
explainability_output_path

### Nettoyer

Enfin, n'oubliez pas de nettoyer les ressources que nous avons configurées et utilisées pour cette démo !

In [ ]:
session.delete_model(model_name)